In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import imageio.v3 as iio
%load_ext autoreload
%autoreload 2
from time import sleep

<h1> WrapUp

<h2> First we create the csv file with the correct columns

In [2]:
data = [[0,0,0,0,0,0,0,0]]
data = pd.DataFrame(data, columns =["UID","Specie","SubSpecie","Distribution","href","Genus","Genus_Info","Value"])

In [3]:
#data.to_csv("data/orchids.csv",index=False, header=True)

<h2> We check for the connection to the web page

In [4]:
base_url = "https://www.worldplants.de/world-orchids/orchid-list"
r = requests.get(base_url)
# print request object
print(r.url)

# print status code
print(r.status_code)

https://www.worldplants.de/world-orchids/orchid-list
200


<h2>We create the list Genre and Value

In [5]:
# Parsing the HTML
soup = BeautifulSoup(r.content, 'html.parser')
 
s = soup.find('select', id='genus')
results = s.find_all('option')
genus = []
values = []
for result in results:
    genus.append(result.text)
    values.append(result.get('value'))

genus.remove("...")
values.remove("0")

In [6]:
gen = pd.Series(genus)
val = pd.Series(values)

In [7]:
genus = pd.DataFrame(gen, columns=["Genus"])
genus["Value"] = val
genus.head()
genus.to_csv("data/genus2.csv", index=False, header=True)

In [8]:
dada =

SyntaxError: invalid syntax (3813348019.py, line 1)

<h2> Functions to extract the data

In [ ]:
def connection_to_webpage(base_url,value):
# we generate the url
    url = base_url + "#g-"+value
    browser = webdriver.Chrome()

    browser.get(url)
    wait = WebDriverWait(browser,20)
    html = browser.page_source

    return browser


In [ ]:
def get_genus(browser):
    # We get distribution information on the Genus
    genus_info = browser.find_elements(By.CSS_SELECTOR, '.genusInfo')
    dis = []
    distribution =""
    for e in genus_info:
        text = e.text
        dis = text.split("\n")
    for i in dis:
        if "Distribution:" in i:
            distribution = i
    return distribution

In [ ]:
def get_species(browser):
    sleep(2)
    orquid_selection = browser.find_elements(By.CSS_SELECTOR, ".type-species")
    name = []
    dist = []
    href = []
    h = []
    data = []
    distribution =[]
    for i in orquid_selection:
        u_id =int(i.get_attribute("id").split("-")[1])
        n = i.find_element(By.CSS_SELECTOR, ".latName").text
        typ = "specie"
        name.append([u_id,n,typ])
        tmp = i.find_elements(By.CSS_SELECTOR, ".distCountry")
        for j in tmp:
            dist.append(j.text)
        distribution.append(dist)
        dist = []
        image = i.find_elements(By.CSS_SELECTOR, "div.imageBox a")
        for j in image:
            tmp =j.get_attribute("href")
            if tmp not in h:
                h.append(tmp)    
        href.append([u_id,h])
        h = []
        data.append([u_id,n,distribution,h])
    return data

In [ ]:
def get_sub_species(browser):
    sleep(2)
    orchid_sub = browser.find_elements(By.CSS_SELECTOR, ".type-subspecies")
    name_sub = []
    dist_sub = []
    href_sub = []
    h = []
    data_sub = []
    distribution = []
    for i in orchid_sub:
        u_id =int(i.get_attribute("id").split("-")[1])
        n = i.find_element(By.CSS_SELECTOR, ".latName").text
        name_sub.append(u_id)
        tmp = i.find_elements(By.CSS_SELECTOR, ".distCountry")
        for j in tmp:
            dist_sub.append([j.text])
        distribution.append(dist_sub)
        dist_sub = []
        image = i.find_elements(By.CSS_SELECTOR, "div.imageBox a")
        for j in image:
            tmp =j.get_attribute("href")
            if tmp not in h:
                h.append(tmp)
        href_sub.append(h)
        data_sub.append([u_id,n,distribution,h])
        h = []
    return data_sub, name_sub

In [ ]:
def get_species_sub_species(name_sub,data,data_sub):
    j = 0
    k = 0
    spe = ""
    for i in name_sub:
        while k == 0:
            if (i-j) in data["UID"].values:
                spe = data[data["UID"] == (i-j)]["Specie"].unique()
                data_sub.loc[data_sub["UID"]==i,"Specie"] = spe
                k = 1
            j += 1
        print("out of while", i)
        j = 0
        k = 0    
    return data_sub

In [ ]:
def create_images(data):
    for i in data.index:
        m = data["href"][i]
        name = data["Specie"][i]
        sub = data["SubSpecie"][i]
        if len(m) != 0:
            for j,k in enumerate(m):             
                tmp = iio.imread(k)
                iio.imwrite(f"images/{name}-{sub}-{j}.jpg",tmp)
                sleep(1)
    return "Images created successfully"

In [ ]:
def save_data(data):
    data = data[["UID","Specie","SubSpecie","Distribution","href","Genus","Genus_Info","Value"]]
    data.to_csv('data/orchids.csv',mode='a', index=False, header=False)
    return "Data appended successfully."

In [ ]:
def add_columns(g, g_d,v,data):
    data["Genus"] = g
    data["Genus_Info"] = g_d
    data["Value"] = v
    sleep(1)
    return data

In [ ]:
def make_df(data,columns,extra):
    df = pd.DataFrame(data, columns =columns)
    df[extra] = ""
    sleep(1)
    return df

<h2> The main program

In [ ]:
for i in range(725,len(genus)):
    v = genus["Value"][i]
    g = genus["Genus"][i]
    url = base_url + "#g-"+v
    browser = webdriver.Chrome()
    browser.get(url)
    html = browser.page_source
    genus_distribution = get_genus(browser)
    print(genus_distribution)
    data = get_species(browser)
    data = make_df(data,["UID","Specie","Distribution","href"],"SubSpecie")
    data_sub, name_sub= get_sub_species(browser)
    wait = WebDriverWait(browser,20)
    data_sub = make_df(data_sub, ["UID","SubSpecie","Distribution","href"], "Specie")
    data_sub = get_species_sub_species(name_sub,data,data_sub)
    df = pd.concat([data,data_sub],ignore_index = True)
    df = add_columns(g, genus_distribution,v,df)
    save_data(df)
    create_images(df)
    sleep(15)
    browser.close()
    print(f"row {i}")